In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import scipy.special as sc
from scipy.stats import weibull_min
import pandas as pd
import os
import seaborn as sns
import sys

np.set_printoptions(threshold=sys.maxsize)

In [3]:
def weib_cdf(x,alpha,gamma):
    return (1 - np.exp(-np.power(x/alpha,gamma)))

In [4]:
# def plot_weibull(mean, std_dev, temp, popt_weibull):
#     #fig=plt.figure(figsize=(16, 14), dpi= 80, facecolor='w', edgecolor='k')
#     fig, ax = plt.subplots(figsize=(10, 4), dpi= 80, facecolor='w', edgecolor='k')

#     '''Textbox with mu and sigma'''
#     textstr = '\n'.join((
#         r'$\mu=%.2f$' % (mean, ),
#         r'$\sigma=%.2f$' % (std_dev, )))

#     # these are matplotlib.patch.Patch properties
#     props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
#     # place a text box in upper left in axes coords
#     ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
#             verticalalignment='top', bbox=props)


#     y_weib=weib_cdf(temp,popt_weibull[0],popt_weibull[1])
#     error_weib=np.power(y_weib-np.squeeze(F),2)
#     plt.plot(temp,y_weib,'r',linewidth=5,label="Weibull Distribution")
#     plt.plot(temp,F,'b',linewidth=5,label="K-M stats")
#     plt.legend(loc=4)
#     plt.ylim(0,1)
#     #label="Alpha "+str(dataset[sample,1])+" Rho "+str(dataset[sample,2])+" Time of First Passage for "+str(censored)+"/"+str(uncensored)+" censored values"
#     label = figLabel
#     plt.title(label)
#     plt.xlabel("Number of time steps")
#     plt.ylabel("Synchronisation probability")
# #     plt.show()
#     plt.savefig(figPath)
#     plt.close(fig)

In [5]:
def load_pd_times(dirPath, experiment_type):
    if(experiment_type!="times"):
        print("experiment_type could be only $times")
        exit(-1)
    
    num_experiment = len([name for name in os.listdir(dirPath) if (os.path.isfile(os.path.join(dirPath, name)) and (name.endswith('position.tsv')))])
    
    if(os.path.exists(dirPath+"/"+experiment_type+".pkl")):
#         print("Sto cazzo di file esiste")
        return (num_experiment,pd.read_pickle(dirPath+"/"+experiment_type+".pkl"))
    
    print("Generating pickle times file")
    df = pd.DataFrame()
    for filename in os.listdir(dirPath):
        if filename.endswith('time_results.tsv'):
            df_single = pd.read_csv(dirPath+"/"+filename, sep="\t")
            df = df.append(df_single)
    
    df.to_pickle(dirPath+"/"+experiment_type+".pkl")
    return (num_experiment,df)

In [6]:
# path = "/home/luigi/Documents/scripts/test_scripts/results/results_2020-02-17_robots_10/2020-02-14_robots#10_alpha#2.0_rho#0.0_experiment_1800/times.pkl"
# df = pd.read_pickle(path)

In [7]:
# df

In [8]:
def evaluate_convergence_time(times):
    conv_times = np.zeros(times.shape[0])
#     print("Time shape", times.shape)
    for idx, elem in enumerate(times):
        if(elem[0] == 0):
            conv_times[idx] = elem[1]
        else:
            conv_times[idx] = elem.min()
    #c_time in ticks 
#     conv_time_batch = np.append(conv_time_batch, conv_times.max()) 
    return conv_times

In [9]:
#data = time vector
#censored = number o missing values
def KM_estimator(data, censored):
    '''K-M estimator'''
    n_est=np.asarray(range(0,data.size))[::-1] + censored  #array from 29 to 0
    RT_sync=[]
    for i in range(n_est.size):
        if len(RT_sync)==0:
            RT_sync.append((n_est[i]-1)/n_est[i])
        else:
            RT_sync.append(RT_sync[-1]*((n_est[i]-1)/n_est[i]))
#     print(RT_sync)
    F=1-np.asarray(RT_sync).reshape(-1,1)
#     print(F)
    return F

In [10]:
def weibull_plot(mean, std_dev, times_value, popt_weibull, F, figLabel, figPath):
    fig, ax = plt.subplots(figsize=(20, 8), dpi= 160, facecolor='w', edgecolor='k')
    '''Textbox with mu and sigma'''
    textstr = '\n'.join((
        r'$\mu=%.2f$' % (mean, ),
        r'$\sigma=%.2f$' % (std_dev, )))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    # place a text box in upper left in axes coords
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)


    y_weib=weib_cdf(times_value,popt_weibull[0],popt_weibull[1])
    error_weib=np.power(y_weib-np.squeeze(F),2)
    plt.plot(times_value,y_weib,'r',linewidth=5,label="Weibull Distribution")
    plt.plot(times_value,F,'b',linewidth=5,label="K-M stats")
    plt.legend(loc=4)
    plt.ylim(0,1)
    #label="Alpha "+str(dataset[sample,1])+" Rho "+str(dataset[sample,2])+" Time of First Passage for "+str(censored)+"/"+str(uncensored)+" censored values"
    label = figLabel
    plt.title(label)
    plt.xlabel("Number of time steps")
    plt.ylabel("Synchronisation probability")
    #     plt.show()
    plt.savefig(figPath)
    plt.close(fig)

In [11]:
def plot_heatmap(dictionary, title, storage_dir):
    for key, value in dictionary.items():
        fig=plt.figure(figsize = (12, 8), dpi=80)
        dataFrame=pd.DataFrame.from_dict(value)
        reversed_df=dataFrame.iloc[::-1]
        ax=sns.heatmap(reversed_df, annot = True, fmt = ".2e", cmap="viridis")
        ax.set_title(title+", num_robots:%s" % (key))
        ax.set_ylabel("alpha")
        ax.set_xlabel("rho")
#         plt.show()
        #Salva su file
        file_name=title+"_%s_robots.png" % (key)
        plt.savefig(storage_dir+'/'+file_name)
    #     reversed_df.to_pickle(file_name[:-4] + ".pickle")
        plt.close(fig)

### Test Script

In [43]:
if __name__ == '__main__':
    main()

Convergence Time alpha
{'100': {'1.6': {}, '2.0': {}, '1.2': {}}, '50': {'1.6': {}, '1.2': {}, '2.0': {}}, '20': {'1.2': {}, '1.6': {}, '2.0': {}}, '10': {'2.0': {}, '1.2': {}, '1.6': {}}}
FPT alpha
{'100': {'1.6': {'0.3': [21684.8781537108, 427.32958630277386], '0.6': [17287.11334243398, 311.5570850275546], '0.0': [27133.62455448213, 607.0723188708473], '0.9': [14317.183134220051, 240.9130443683136]}, '2.0': {'0.9': [14808.394111724256, 254.53779141504526], '0.3': [27763.61467317399, 633.3981227010826], '0.0': [34771.71881905645, 866.2500405557003], '0.6': [20108.588556226896, 400.90400114246074]}, '1.2': {'0.6': [16526.983346059966, 293.54973424819093], '0.0': [21035.5808624603, 418.37762263949634], '0.3': [18804.234139656477, 357.00974229304006], '0.9': [14736.055672880886, 252.1535946578318]}}, '50': {'1.6': {'0.3': [19439.712875129695, 552.6226063970992], '0.6': [15610.447982193726, 400.07062415703956], '0.9': [12417.354121204138, 291.492883709171], '0.0': [23330.43057162276, 698.

In [42]:
def main():
    robots = [10, 20, 50, 100]
    mean_fpt_dict = dict()
    convergence_time_dict = dict()
    #WARNING : you can remove convergence_time_alpha_dict using just convergence_time_dict
    convergence_time_alpha_dict = dict()
    mean_fpt_dict_alpha = dict()
    #When conv_time_estimation==Flase -> fpt estimation
    conv_time_estimation = False
    bound_is=75000

    from datetime import date
    today = date.today()

    main_folder = "./results"
    folder_experiments = "2020-05-25_simple_collision_avoidance_experiment"
    folder = os.path.join(main_folder, folder_experiments)
    script_dir = os.path.abspath('')
    results_dir = os.path.join(script_dir, 'Plots/'+folder_experiments+'/Weibull')

    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)


    conv_time_dir = os.path.join(results_dir, 'convergence_time')

    ftp_dir = os.path.join(results_dir, 'first_passage_time')

    if not os.path.isdir(conv_time_dir):
        os.makedirs(conv_time_dir)
    if not os.path.isdir(ftp_dir):
        os.makedirs(ftp_dir)
        
#     for r in robots:    
#     folder = script_dir+"/results/results_2020-02-21_rob_%s" %(r)
    #folder = script_dir+"/results/2020-05-25_simple_collision_avoidance_experiment/100_robots"

    if not os.path.isdir(folder):
        print("folder is not an existing path")
        exit(-1)


    for dirName, subdirList, fileList in os.walk(folder):
        num_robots = "0"
        rho = -1.0
        alpha = -1.0
        elements=dirName.split("_")
        for e in elements:
            if e.startswith("robots"):
                num_robots=e.split("#")[-1]
                if(num_robots not in mean_fpt_dict):
                    mean_fpt_dict[num_robots]=dict()
                    convergence_time_dict[num_robots]=dict()
                    convergence_time_alpha_dict[num_robots]=dict()
                    mean_fpt_dict_alpha[num_robots]=dict()

            if(e.startswith("rho")):
                rho=float(e.split("#")[-1])
            if(e.startswith("alpha")):
                alpha=float(e.split("#")[-1])

        if(num_robots == "0" or rho == -1.0 or alpha == -1):
            continue

    #     print(dirName)
    #     print(num_robots)
    #     print(mean_fpt_dict)

        rho_str=str(rho)
        alpha_str=str(alpha)
    #     print("rho", rho_str)
    #     print("alpha", alpha_str)
        if(rho_str not in mean_fpt_dict[num_robots]):
            mean_fpt_dict[num_robots][rho_str]=dict()
    #         print(mean_fpt_dict)

            convergence_time_dict[num_robots][rho_str]=dict()

        if(alpha_str not in convergence_time_alpha_dict[num_robots]):
            convergence_time_alpha_dict[num_robots][alpha_str]=dict()
            mean_fpt_dict_alpha[num_robots][alpha_str]=dict()
    #         print(total_dict)
        #WARNING : di mettere alpha probabilmente non ce n'è bisogno
    #     if(alpha_str not in total_dict[num_robots][rho_str]):
    #         total_dict[num_robots][rho_str][alpha_str]=dict()
    #         mean_fpt_dict[num_robots][rho_str][alpha_str]=dict()
    #         convergence_time_dict[num_robots][rho_str][alpha_str]=dict()


        (num_experiment, df) = load_pd_times(dirName, "times")


        df_times = df.values[:,1:]
        convergence_times = evaluate_convergence_time(df_times)
    #     print(dirName)
    #     print(convergence_times.shape)


    #     print(df_times.shape)
    #     print("num experiments: ", num_experiment)


        if(conv_time_estimation):
            '''Weibull distribution for Convergence Time'''

            #get the time in whitch each robot has at least info about the target
            convergence_time_batches = np.amax(convergence_times.reshape(num_experiment,-1) , axis=1) 
    #         print(dirName)

            #order convergence_time_batches in increasing order
            convergence_time_batches = convergence_time_batches[np.argsort(convergence_time_batches)]
    #         print(convergence_time_batches.shape)
    #         print(convergence_time_batches)
            figPath = conv_time_dir+'/'+"conv_time_robots_%s_alpha_%s_rho_%s.png" % (num_robots, alpha_str, rho_str) 
            figLabel = "Convergence Time robots:%s alpha:%s, rho:%s.png" % (num_robots, alpha_str, rho_str)
    #         censored = 1
            censored = convergence_time_batches.size - np.count_nonzero(convergence_time_batches)
            if (censored):
                times_value = convergence_time_batches[censored:].reshape(-1)
            else:
                censored = 1
                times_value = convergence_time_batches.reshape(-1)

            F = KM_estimator(times_value, censored)

            #popt_weibull[0] is alpha
            #popt_weibull[1] is gamma    
            popt_weibull,_= curve_fit(weib_cdf,xdata=times_value,ydata=np.squeeze(F),bounds=(0,[bound_is,10]),method='trf')
            mean = sc.gamma(1+(1./popt_weibull[1]))*popt_weibull[0]
        #     print("mean",mean)
            std_dev = np.sqrt(popt_weibull[0]**2 * sc.gamma(1+(2./popt_weibull[1])) - mean**2)

            std_error = std_dev / np.sqrt(times_value.size)
            convergence_time_alpha_dict[num_robots][alpha_str][rho_str]= [mean,std_error] 
            convergence_time_dict[num_robots][rho_str][alpha_str]= mean
        #     print(times_value.shape)
            weibull_plot(mean, std_dev, times_value, popt_weibull, F, figLabel, figPath)
            plot_heatmap(convergence_time_dict, "Convergence Time", conv_time_dir)

        else:
            ''' Weibull distribution for First Passage Time'''
            figPath = ftp_dir+'/'+"fpt_robots_%s_alpha_%s_rho_%s.png" % (num_robots, alpha_str, rho_str)
            figLabel = "fpt robots:%s alpha:%s, rho:%s.png" % (num_robots, alpha_str, rho_str)
            fpt = df.values[:,1:2]
            censored = fpt.size - np.count_nonzero(fpt)
            fpt = fpt[np.argsort(fpt.reshape(-1))]
            times_value = fpt[censored:].reshape(-1)


            F = KM_estimator(times_value, censored)

            #popt_weibull[0] is alpha
            #popt_weibull[1] is gamma    
            popt_weibull,_= curve_fit(weib_cdf,xdata=times_value,ydata=np.squeeze(F),bounds=(0,[bound_is,10]),method='trf')
            mean = sc.gamma(1+(1./popt_weibull[1]))*popt_weibull[0]
            mean_fpt_dict[num_robots][rho_str][alpha_str] = mean
        #     print("mean",mean)
            std_dev = np.sqrt(popt_weibull[0]**2 * sc.gamma(1+(2./popt_weibull[1])) - mean**2)

            std_error = std_dev / np.sqrt(times_value.size)
            mean_fpt_dict_alpha[num_robots][alpha_str][rho_str]= [mean,std_error] 

        #     print(times_value.shape)
            weibull_plot(mean, std_dev, times_value, popt_weibull, F, figLabel, figPath)
            plot_heatmap(mean_fpt_dict,"Average First Passage Time", ftp_dir)

#     print("Convergence Time")
#     print(convergence_time_dict)
#     print("Average First Passage Time")
#     print(mean_fpt_dict)
    print("Convergence Time alpha")
    print(convergence_time_alpha_dict)
    print("FPT alpha")
    print(mean_fpt_dict_alpha)

In [ ]:
plot_heatmap(mean_fpt_dict,"Average First Passage Time")

In [ ]:
plot_heatmap(convergence_time_dict, "Convergence Time")

In [ ]:
convergence_time_dict

In [ ]:
mean_fpt_dict